## Scikit-Learn (Sklearn) Course

<span>
0. sklearn workflow overview<br>
1. preparing data (exploring, cleaning, transforming, reducing, splitting)<br>
2. selecting machine learning model / algorithm<br>
3. training algorithm and making predictions<br>
4. evaluating algorithm<br>
<span style="color:orange">5. improving model</span><br>
6. saving and loading algorithm<br>
7. putting it all together
</span>

## 5. Improving the Model

#### General concepts

--- baseline  
first model = baseline model  
first prediction = baseline prediction

--- improving model / data perspective  
collecting more data (the more data the better)  
improving data > adding more features

--- improving model / algorithm perspective  
using a better, more complex algorithm  
improving current algorithm with hyperparameter tuning

--- hyperparameters  
settings of the algorithm that the user can adjust  
basically, hyperparameters are the function parameters of the algorithm instance  
hyperparameters are detailed in the documentation of each algorithm

--- hyperparameter adjustment methods  
by hand (guessing)  
random search with RandomSearchCV() algorithm  
exhaustive search (brute force) with GridSeachCV() algorithm

#### Preparing data

In [ ]:
### imports
import numpy, pandas

In [ ]:
### preparing data

### loading heart disease data into dataframe
heart_disease = pandas.read_csv("data-heart-disease.csv")

### shuffling heart disease dataframe
numpy.random.seed(42)
heart_disease = heart_disease.sample(frac=1.0)

### splitting data features <> target
features = heart_disease.drop(columns="target")
target = heart_disease.loc[:, "target"]

### splitting data train <> test
train_index = round(0.7 * heart_disease.index.size)
valid_index = round(0.85 * heart_disease.index.size)
features_train, target_train = features[:train_index], target[:train_index]
features_valid, target_valid = features[train_index:valid_index], target[train_index:valid_index]
features_test, target_test = features[valid_index:], target[valid_index:]

#### Tuning hyperparameters by hand

In [ ]:
### imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
### classification algorithm evaluation function
def evaluatePreds(target_preds, target_valid):
    metric_dict = {
        "accuracy": accuracy_score(y_pred=target_preds, y_true=target_valid),
        "precision": precision_score(y_pred=target_preds, y_true=target_valid),
        "recall": recall_score(y_pred=target_preds, y_true=target_valid),
        "f1": f1_score(y_pred=target_preds, y_true=target_valid)}
    print(f"""Accuracy: {100.0 * metric_dict["accuracy"]:.3f}%""")
    print(f"""Precision: {100.0 * metric_dict["precision"]:.3f}%""")
    print(f"""Recall: {100.0 * metric_dict["recall"]:.3f}%""")
    print(f"""F1 Score: {100.0 * metric_dict["f1"]:.3f}%""")
    return

In [ ]:
### creating and evaluating baseline algorithm

### creating, training, predicting algorithm
numpy.random.seed(42)
classifier = RandomForestClassifier()
classifier.fit(X=features_train, y=target_train)
target_preds = classifier.predict(X=features_valid)

### evaluating algorithm
evaluatePreds(target_preds, target_valid)

In [ ]:
### reading default hyperparameters of baseline algorithm
classifier.get_params()

--- hyperparameters to adjust  
`max_depth=`  
`max_features=`  
`min_samples_leaf=`  
`min_samples_split=`  
`n_estimators=`

In [ ]:
### creating and evaluating adjusted algorithm (max_depth=10)

### creating, training, predicting algorithm
numpy.random.seed(42)
classifier = RandomForestClassifier(max_depth=10)
classifier.fit(X=features_train, y=target_train)
target_preds = classifier.predict(X=features_valid)

### evaluating algorithm
evaluatePreds(target_preds, target_valid)

In [ ]:
### creating and evaluating adjusted algorithm (n_estimators=500)

### creating, training, predicting algorithm
numpy.random.seed(42)
classifier = RandomForestClassifier(n_estimators=500)
classifier.fit(X=features_train, y=target_train)
target_preds = classifier.predict(X=features_valid)

### evaluating algorithm
evaluatePreds(target_preds, target_valid)

#### Tuning hyperparameters with randomized search cross validation

In [ ]:
### imports
import numpy, pandas
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
### preparing data

### loading heart disease data into dataframe
heart_disease = pandas.read_csv("data-heart-disease.csv")

### splitting data features <> target
features = heart_disease.drop(columns="target")
target = heart_disease.loc[:, "target"]

### splitting data train <> test
numpy.random.seed(42)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

In [ ]:
### running randomized search

### creating search grid
search_grid = {
    "max_depth": [None, 5, 10, 20, 30],
    "max_features": ["sqrt"],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 4, 6],
    "n_estimators": [10, 100, 200, 500, 1000, 1200]}

### creating randomized search algorithm
numpy.random.seed(42)
rscv_classifier = RandomizedSearchCV(
    estimator=RandomForestClassifier(n_jobs=-1),
    param_distributions=search_grid,
    n_iter=10, cv=5, verbose=2)

### training randomized search algorithm
rscv_classifier.fit(X=features, y=target);

In [ ]:
### reading best parameters
rscv_classifier.best_params_

In [ ]:
### evaluating algorithm
target_preds = rscv_classifier.predict(features_test)
evaluatePreds(target_preds, target_test)

#### Creating regression model

In [ ]:
### imports
import numpy, pandas
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
### preparing data

### loading california housing dataset
housing_dict = fetch_california_housing()

### creating california housing dataframe
housing_df = pandas.DataFrame(data=housing_dict["data"], columns=housing_dict["feature_names"])
housing_df["MedHouseVal"] = housing_dict["target"]

### splitting data features/target
features = housing_df.drop(columns="MedHouseVal")
target = housing_df.loc[:, "MedHouseVal"]

### splitting data train/test
numpy.random.seed(42)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)
target_test: numpy.ndarray

In [ ]:
### random forest regressor training and prediction

### instantiating model
numpy.random.seed(42)
regressor = RandomForestRegressor(n_estimators=100)

### training model / prediction
regressor.fit(X=features_train, y=target_train)
target_preds = regressor.predict(X=features_test)

#### Evaluating regression model

In [ ]:
### imports
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

--- `.score()` method  
default `.score()` and cross validation metric for regression models is [coefficient of determination (r^2)](https://en.wikipedia.org/wiki/Coefficient_of_determination)

In [ ]:
### evaluating algorithm with .score() method on test data
regressor.score(X=features_test, y=target_test)

--- coefficient of determination (r2-score)  
when a model predicts the mean of targets, its r2-score is 0.0  
when a model perfectly predicts all targets, its r2-score is 1.0  
`numpy.full(shape=, fill_value=)` creates an array of `shape=` filled with `fill_value=`

In [ ]:
### getting r2-score of 0.0
target_test_mean = numpy.full(shape=len(target_test), fill_value=target_test.mean())
r2_score(y_true=target_test, y_pred=target_test_mean)

In [ ]:
### getting r2-score of 1.0
r2_score(y_true=target_test, y_pred=target_test)

In [ ]:
### evaluating algorithm with cross-validated default score (r2-score)
numpy.random.seed(42)
cv_r2 = cross_val_score(estimator=regressor, X=features, y=target, cv=3, scoring=None)
cv_r2.mean()

--- mean absolute error (MAE)  
mean of absolute differences between predictions and targets  
represents the linear magnitude of prediction error

In [ ]:
### computing mean absolute error with sklearn function
mean_absolute_error(y_true=target_test, y_pred=target_preds)

In [ ]:
### computing mean absolute error step-by-step
error_df = pandas.DataFrame(data={"target preds": target_preds, "target test": target_test})
error_df["differences"] = numpy.abs(error_df["target preds"] - error_df["target test"])
error_df["differences"].mean()

In [ ]:
### evaluating algorithm with cross-validated mean absolute error
numpy.random.seed(42)
cv_mae = cross_val_score(estimator=regressor, X=features, y=target, cv=3, scoring="neg_mean_absolute_error")
cv_mae.mean()

--- mean squared error (MSE)  
mean of squared differences between predictions and targets  
squaring emphasizes large errors and diminishes small errors  
there is also root mean squared error (RMSE) - see sklearn documentation

In [ ]:
### computing mean squared error with sklearn function
mean_squared_error(y_true=target_test, y_pred=target_preds)